In [1]:
# coding:utf-8
import pandas as pd 
import pickle 
import pynlpir
import numpy as np
from collections import Counter
import jieba 
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.cluster import KMeans
import json
import xlrd
import time
import pickle
import random

In [2]:
data = xlrd.open_workbook('data/知识点导出_2018-10-24.xlsx')  #读取数据
page = len(data.sheets())                             #获取sheet的数量
table = data.sheets()[0]                                #打开第一张表
nrows = table.nrows                                     #获取总行数
ncols = table.ncols                                       #获取总列数   
gold_data = []
knowledge = []
for i in range(1,nrows):
    know, ques = table.row_values(i)
    gold_data.append(ques)
    knowledge.append(know)

In [3]:
add_data = open('data/msgwyeth.txt', 'r').read().split('\n')
print knowledge[0]
len(add_data),len(knowledge)

你好


(501393, 1750)

In [4]:
def get_question(data, knowledge):
    questions = []
    knowledges = []
    for i in range(len(data)):
        ques = data[i].split('||')
        for que in ques:
            if que != '':
                questions.append(que)
                knowledges.append(knowledge[i])
    print questions[0]
    print len(questions)
    return questions, knowledges

In [5]:
gold_questions, knowledges = get_question(gold_data, knowledge)

亲在么
74936


In [6]:
#text = gold_questions + add_data #list(set(list(data['match'])))
def jieba_tokenize(text):
    return jieba.lcut(text) 

def cluster(text):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=jieba_tokenize, \
    lowercase=False)
    '''
    tokenizer: 指定分词函数
    lowercase: 在分词之前将所有的文本转换成小写，因为涉及到中文文本处理，
    所以最好是False
    '''
    text_list = text
     #需要进行聚类的文本集
    tfidf_matrix = tfidf_vectorizer.fit_transform(text_list)

    num_clusters = len(knowledge)
    print num_clusters
    km_cluster = KMeans(n_clusters=num_clusters, max_iter=100, n_init=40, \
                        init='k-means++',n_jobs=20)
    '''
    n_clusters: 指定K的值
    max_iter: 对于单次初始值计算的最大迭代次数
    n_init: 重新选择初始值的次数
    init: 制定初始值选择的算法
    n_jobs: 进程个数，为-1的时候是指默认跑满CPU
    注意，这个对于单个初始值的计算始终只会使用单进程计算，
    并行计算只是针对与不同初始值的计算。比如n_init=10，n_jobs=40, 
    服务器上面有20个CPU可以开40个进程，最终只会开10个进程
    '''
    #返回各自文本的所被分配到的类索引
    result = km_cluster.fit_predict(tfidf_matrix)

    print "Predicting result: ", result
    return result

In [ ]:
for i in range(50):
    print i
    start = time.time()
    sample_add = random.sample(add_data, i * 10000)  #从list中随机获取i * 10000个元素，作为一个片断返回
    text = gold_questions + sample_add
    print len(text)
    result = cluster(text)
    pickle.dump(result, open('data/add'+str(i*10000)+'sample_cluster.txt', 'w'))
    end = time.time()
    print 'take time :' + str(end - start) + 'to cluster ' + str(len(text)) + 'data'

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


0
74936


Loading model cost 0.276 seconds.
Prefix dict has been built succesfully.


1750


/home/works/dl-tools/anaconda2/envs/tfenv.v1.2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [ ]:
df = pd.DataFrame({'request_word':text[:len(gold_questions)], 'predict':result, 'real': knowledges})
new_df = {}
new_df['class_num'] = []
new_df['request_words'] = []
c = 0
count = 0
new_texts = []
class_ = []
num_ = []
for i in df['predict'].unique():
    d = list(df[df['predict'] == i]['request_word'])
    new_df['class_num'].append(c)
    new_df['request_words'].append('||'.join(d))
    new_texts.append('||'.join(d))
    num_.append(len(d))
    class_.append(d[0])
    count += len(d)
    c += 1
dff = pd.DataFrame({'question':new_texts, 'class':class_, 'num':num_})

In [ ]:
dff

In [ ]:
from sklearn import metrics
labels_true = knowledges
labels_pred = result
 
print (metrics.adjusted_rand_score(labels_true, labels_pred))
